In [50]:
import numpy as np
import pandas as pd

import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
train = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/train2.csv')
test = pd.read_csv('/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/test2.csv')

In [29]:
print('train dataset has {} rows and {} columns.'.format(train.shape[0],train.shape[1]))
print('test dataset has {} rows and {} columns.'.format(test.shape[0],test.shape[1]))

train dataset has 1754884 rows and 124 columns.
test dataset has 113640 rows and 122 columns.


In [30]:
train = train[train['label']!=-1]

In [34]:
train['weekday'] = train['weekday'].astype('int').astype('category')
test['weekday'] = test['weekday'].astype('int').astype('category')

In [35]:
train = pd.get_dummies(train, columns=['weekday'])
test = pd.get_dummies(test, columns=['weekday'])

# 建模

In [37]:
features = [x for x in train.columns if x not in ['User_id','Merchant_id','Coupon_id','Date_received','Discount_rate','Date','label','day']]
test_preds = test[['User_id','Coupon_id','Date_received']]
train = xgb.DMatrix(train[features],label=train['label'])
test = xgb.DMatrix(test[features])

In [38]:
params={'booster':'gbtree',
	    'objective': 'rank:pairwise',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

In [ ]:
#train on dataset1, evaluate on dataset2
#watchlist = [(dataset1,'train'),(dataset2,'val')]
#model = xgb.train(params,dataset1,num_boost_round=3000,evals=watchlist,early_stopping_rounds=300)

watchlist = [(train,'train')]
model = xgb.train(params,train,num_boost_round=3500,evals=watchlist,early_stopping_rounds=50)

#predict test set
test_preds['Probability'] = model.predict(test)
test_preds.Probability = MinMaxScaler().fit_transform(test_preds.Probability.values.reshape(-1, 1))
test_preds.sort_values(by=['Coupon_id','Probability'],inplace=True)
test_preds.to_csv("/content/drive/My Drive/Case/O2O Coupon Use Prediction/data/Mysubmission3.csv",index=None)
print(test_preds.describe())

In [70]:
test_preds

,User_id,Coupon_id,Date_received,Probability
93199,2907482,3,2016-07-04,0.013243
25264,1647633,3,2016-07-10,0.019260
98688,1368563,3,2016-07-22,0.019745
54554,1462435,3,2016-07-08,0.021567
89443,5155319,3,2016-07-03,0.023099
...,...,...,...,...
91245,3890474,14035,2016-07-17,0.042594
67678,1506889,14036,2016-07-01,0.192209
43720,4867178,14040,2016-07-30,0.166602
86625,5953319,14040,2016-07-18,0.194088
